In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls

drive  sample_data


In [4]:
%cd drive/My Drive/BERT

/content/drive/My Drive/BERT


In [5]:
import pytorch_pretrained_bert

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


The code block below trains the model for a few epochs, I monitored the accuracy and selected 2 epochs to finetune the model to prevent overfitting

In [0]:
!python train.py --data_dir data/task1/ --bert_model_dir model/ --model_dir experiments/base_model

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
device: cuda, n_gpu: 1, 16-bits training: False
Loading the datasets...
loading vocabulary file model/vocab.txt
loading archive file model/
Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 31090
}

Weights of BertForTokenClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
Weights from pretrained model not used in BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.we

Importing required libraries


In [0]:
import random
import numpy as np
import os
import sys

import torch

from pytorch_pretrained_bert import BertTokenizer, BertForTokenClassification, BertConfig

import utils

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


custom dataloader to load sentences and feed to model in batches

In [0]:
class DataLoader(object):
    def __init__(self, data_dir, bert_model_dir, max_len, device, token_pad_idx=0):
        self.data_dir = data_dir
        # self.batch_size = params.batch_size
        self.max_len = max_len
        self.device = device
        self.seed = 23
        self.batch_size = 4
        self.token_pad_idx = 0

        # tags = self.load_tags()
        # self.tag2idx = {tag: idx for idx, tag in enumerate(tags)}
        # self.idx2tag = {idx: tag for idx, tag in enumerate(tags)}
        # params.tag2idx = self.tag2idx
        # params.idx2tag = self.idx2tag
        self.tokenizer = BertTokenizer.from_pretrained(bert_model_dir, do_lower_case=True)

    def ret_tags(self):
        tags = self.load_tags()
        self.tag2idx = {tag: idx for idx, tag in enumerate(tags)}
        self.idx2tag = {idx: tag for idx, tag in enumerate(tags)}
        self.tag_pad_idx = self.tag2idx['O']
        return self.tag2idx, self.idx2tag

    def load_tags(self):
        tags = []
        file_path = os.path.join(self.data_dir, 'task1' , 'tags.txt')
        with open(file_path, 'r', encoding="utf-8") as file:
            for tag in file:
                tags.append(tag.strip())
        return tags

    def load_sentences_tags(self, sentences_file, d):
        """Loads sentences and tags from their corresponding files. 
            Maps tokens and tags to their indices and stores them in the provided dict d.
        """
        sentences = []
        # tags = []

        with open(sentences_file, 'r', encoding="utf-8") as file:
            for line in file:
                # replace each token by its index
                tokens = line.split()
                sentences.append(self.tokenizer.convert_tokens_to_ids(tokens))
        
        # with open(tags_file, 'r', encoding="utf-8") as file:
        #     for line in file:
        #         # replace each tag by its index
        #         tag_seq = [self.tag2idx.get(tag) for tag in line.strip().split(' ')]
        #         tags.append(tag_seq)

        # checks to ensure there is a tag for each token
        # assert len(sentences) == len(tags)
        # for i in range(len(sentences)):
#             print(sentences[i], tags[i])
            # assert len(tags[i]) == len(sentences[i])

        # storing sentences and tags in dict d
        d['data'] = sentences
        # d['tags'] = tags
        d['size'] = len(sentences)

    def load_data(self, data_type):
        """Loads the data for each type in types from data_dir.

        Args:
            data_type: (str) has one of 'train', 'val', 'test' depending on which data is required.
        Returns:
            data: (dict) contains the data with tags for each type in types.
        """
        data = {}
        
        if data_type in ['train', 'val', 'test']:
            sentences_file = os.path.join(self.data_dir, 'h1_7.txt')
            # tags_path = os.path.join(self.data_dir, data_type, 'tags.txt')
            self.load_sentences_tags(sentences_file, data)
        else:
            raise ValueError("data type not in ['train', 'val', 'test']")
        return data

    def data_iterator(self, data, shuffle=False):
        """Returns a generator that yields batches data with tags.

        Args:
            data: (dict) contains data which has keys 'data', 'tags' and 'size'
            shuffle: (bool) whether the data should be shuffled
            
        Yields:
            batch_data: (tensor) shape: (batch_size, max_len)
            batch_tags: (tensor) shape: (batch_size, max_len)
        """

        # make a list that decides the order in which we go over the data- this avoids explicit shuffling of data
        order = list(range(data['size']))

        # one pass over data
        for i in range(data['size']//self.batch_size):
            # fetch sentences and tags
            sentences = [data['data'][idx] for idx in order[i*self.batch_size:(i+1)*self.batch_size]]
            # tags = [data['tags'][idx] for idx in order[i*self.batch_size:(i+1)*self.batch_size]]

            # batch length
            batch_len = len(sentences)

            # compute length of longest sentence in batch
            batch_max_len = max([len(s) for s in sentences])
            max_len = min(batch_max_len, self.max_len)

            # prepare a numpy array with the data, initialising the data with pad_idx
            batch_data = self.token_pad_idx * np.ones((batch_len, max_len))
            # batch_tags = self.tag_pad_idx * np.ones((batch_len, max_len))

            # copy the data to the numpy array
            for j in range(batch_len):
                cur_len = len(sentences[j])
                if cur_len <= max_len:
                    batch_data[j][:cur_len] = sentences[j]
                    # batch_tags[j][:cur_len] = tags[j]
                else:
                    batch_data[j] = sentences[j][:max_len]
                    # batch_tags[j] = tags[j][:max_len]

            # since all data are indices, we convert them to torch LongTensors
            batch_data = torch.tensor(batch_data, dtype=torch.long)
            # batch_tags = torch.tensor(batch_tags, dtype=torch.long)

            # shift tensors to GPU if available
            batch_data = batch_data.to(self.device)
            # , batch_tags.to(self.device)
        
            yield batch_data

Model Initialization

In [0]:
json_path = os.path.join('experiments/base_model', 'params.json')
data_dir = os.path.join('data/')
bert_model_dir = os.path.join('model/')
model_dir = os.path.join('experiments/base_model')
restore_file = os.path.join('best')
print(json_path)

# Use GPUs if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set the random seed for reproducible experiments
random.seed(23)
torch.manual_seed(23)

# Initialize the DataLoader
params = utils.Params(json_path)
data_loader = DataLoader(data_dir, bert_model_dir, params.max_len,device, token_pad_idx=0)
tag2idx, idx2tag = data_loader.ret_tags()
print("- done.")

# Define the model
config_path = os.path.join(bert_model_dir, 'bert_config.json')
config = BertConfig.from_json_file(config_path)
model = BertForTokenClassification(config, num_labels=len(tag2idx))

model.to(device)
# Reload weights from the saved file
chkpt = utils.load_checkpoint(os.path.join(model_dir, restore_file + '.pth.tar'), model)
# chkpt['optim_dict']


experiments/base_model/params.json
- done.


Evaluation

In [0]:
# Load data
test_data = data_loader.load_data('test')
# Specify the test set size
test_size = test_data['size']
# params.eval_steps = params.test_size // params.batch_size

data_iterator = data_loader.data_iterator(test_data, shuffle=False)

print("Starting evaluation...")
model.eval()
pred_tags = []
eval_steps = test_size//4

for _ in range(eval_steps):
    # fetch the next evaluation batch
    batch_data = next(data_iterator)
    batch_masks = batch_data.gt(0)
    batch_output = model(batch_data, token_type_ids=None, attention_mask=batch_masks)

    batch_output = batch_output.detach().cpu().numpy()
    # print(np.argmax(batch_output, axis=2))
    for indices in np.argmax(batch_output, axis=2):
      pred_tags.append([idx2tag.get(idx) for idx in indices])

    # pred_tags.append([idx2tag.get(idx) for indices in np.argmax(batch_output, axis=2) for idx in indices])

# test_metrics = evaluate(model, test_data_iterator, params, mark='Test', verbose=True)
print(pred_tags)

Starting evaluation...
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'I', 'I', 'O', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I'], ['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'O', 'O', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'O', 'O', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I'

In [0]:
import numpy as np
np.array(pred_tags).dump(open('results_correct.npy', 'wb'))

In [0]:
len(pred_tags), len(sentences)

(2644, 2645)

In [0]:
sentences_file = os.path.join(data_dir, 'h1_7.txt')
tokenizer = BertTokenizer.from_pretrained(bert_model_dir, do_lower_case=True)

In [0]:
sentences = []
with open(sentences_file, 'r', encoding="utf-8") as file:
  for line in file:
    # replace each token by its index
    tokens = line.split()
    # print(tokens)
    sentences.append(tokens)
    # sentences.append(tokenizer.convert_tokens_to_ids(tokens))

In [0]:
print(sentences[:10])

[['Search', 'APIs'], [], ['Most', 'search', 'APIs', 'are', 'multi-index,', 'with', 'the', 'exception', 'of', 'the', 'Explain', 'API', 'endpoints.'], [], ['Routing'], [], ['When', 'executing', 'a', 'search,', 'Elasticsearch', 'will', 'pick', 'the', '"best"', 'copy', 'of', 'the', 'data', 'based', 'on', 'the', 'adaptive', 'replica', 'selection', 'formula.', 'Which', 'shards', 'will', 'be', 'searched', 'on', 'can', 'also', 'be', 'controlled', 'by', 'providing', 'the', 'routing', 'parameter.', 'For', 'example,', 'when', 'indexing', 'tweets,', 'the', 'routing', 'value', 'can', 'be', 'the', 'user', 'name:'], [], ['In', 'such', 'a', 'case,', 'if', 'we', 'want', 'to', 'search', 'only', 'on', 'the', 'tweets', 'for', 'a', 'specific', 'user,', 'we', 'can', 'specify', 'it', 'as', 'the', 'routing,', 'resulting', 'in', 'the', 'search', 'hitting', 'only', 'the', 'relevant', 'shard:'], []]


Extraction of key phrases

In [0]:
extract = []
with open(sentences_file, 'r', encoding="utf-8") as file:
  i=0
  for line in file:
    # replace each token by its index
    tokens = line.split()
    # print(tokens)
    # sentences.append(tokens)
    temp = []
    j=0
    for token in tokens:
      if pred_tags[i][j]=='I':
        temp.append(token)
        j+=1
    if len(temp)>0:
      extract.append(temp)
    i+=1
    # sentences.append(tokenizer.convert_tokens_to_ids(tokens))

In [0]:
extract[:10]

[['When',
  'executing',
  'a',
  'search,',
  'Elasticsearch',
  'will',
  'pick',
  'the',
  '"best"',
  'copy',
  'of',
  'the',
  'data',
  'based',
  'on',
  'the',
  'adaptive',
  'replica',
  'selection',
  'formula.'],
 ['In'],
 ['The',
  'routing',
  'parameter',
  'can',
  'be',
  'multi',
  'valued',
  'represented',
  'as',
  'a',
  'comma'],
 ['Response',
  'time',
  'of',
  'past',
  'requests',
  'between',
  'the',
  'coordinating',
  'node',
  'and',
  'the',
  'node',
  'containing',
  'the',
  'copy',
  'of',
  'the',
  'data'],
 ['Time', 'past', 'search', 'requests'],
 ['The',
  'queue',
  'size',
  'of',
  'the',
  'search',
  'threadpool',
  'on',
  'the',
  'node',
  'containing',
  'the',
  'data'],
 ['This',
  'can',
  'be',
  'turned',
  'off',
  'by',
  'changing',
  'the',
  'dynamic',
  'cluster',
  'setting',
  'cluster.routing.use_adaptive_replica_selection',
  'from',
  'true',
  'to',
  'false:'],
 ['If',
  'adaptive',
  'replica',
  'selection',
  'is'

In [0]:
with open('extracted.txt', 'w') as file:
  file.writelines(' '.join(str(j) for j in i) + '\n' for i in extract)

In [0]:
len(extract)

525

### The drive link to download extracted text
### https://drive.google.com/file/d/1-9QsDiHQT8CDad8geqPJkueJTnYn2pQg/view?usp=sharing 

# BERT BiLstm class

In [0]:
import torch
import torch.nn as nn

In [0]:
class Net(nn.Module):
    def __init__(self, args, params, device):
        super(Net, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained(args.bert_model_dir, num_labels=len(params.tag2idx))
        self.bilstm = nn.LSTM(bidirectional=True, num_layers=2, input_size=768, hidden_size=768//2, batch_first=True)
        self.fc = nn.Linear(768, len(params.tag2idx))
        self.num_labels=2
        self.device = device
    # model(batch_data, token_type_ids=None, attention_mask=batch_masks, labels=batch_tags)
    def forward(self, x, token_type_ids=None, attention_mask=None, y=None):
        x = x.to(self.device)
        if y is not None:
            y = y.to(self.device)
        # enc = self.bert(x, token_type_ids=None, attention_mask=attention_mask)
        # self.bert.eval()

        with torch.no_grad():
            enc = self.bert(x, token_type_ids=None, attention_mask=attention_mask)
            # enc = encoded_layers[-1]

        enc, _ = self.bilstm(enc)
        logits = self.fc(enc)
        y_hat = logits.argmax(-1)
        if y is not None:
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1,30]).float())
            loss = loss_fct(logits.view(-1, self.num_labels), y.view(-1))
            return loss

        return logits

## The above file has been changed in the orginal code repository because some changes also had to be made in `pytorch-pretrained-bert` library, importing all code would not have been feasable

In [17]:
!python train.py --data_dir data/task1/ --bert_model_dir model/ --model_dir experiments/base_model

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
device: cuda, n_gpu: 1, 16-bits training: False
Loading the datasets...
loading vocabulary file model/vocab.txt
loading archive file model/
Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 31090
}

Weights of BertForTokenClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
Weights from pretrained model not used in BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.we